In [1]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

In [3]:
import re

from typing import List, Union
from langchain import OpenAI
from langchain import LLMChain
from langchain import LLMMathChain
from langchain.tools import DuckDuckGoSearchRun
from langchain.utilities import WikipediaAPIWrapper
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser, initialize_agent
from langchain.prompts import StringPromptTemplate
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema import AgentAction, AgentFinish
from langchain.agents.agent_types import AgentType



In [17]:
llm = OpenAI(temperature=0, model='gpt-3.5-turbo-instruct')

In [18]:
search = DuckDuckGoSearchRun()
wikipedia = WikipediaAPIWrapper()
def wrapper(text):
    search_results = search.run(f"site:nih.gov {text}")
    return search_results

search_tool = Tool(
        name = "Search NIH",
        func = wrapper,
        description = "useful for when you need to answer medical, pharmalogical and biomedical questions"
    )

wiki_tool = Tool(
        name = "Wikipedia",
        func = wikipedia.run,
        description = "useful for searching information about various events worldwide, issues, dates, years etc. Use precise information"
    )

llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
math_tool = Tool.from_function(
    func = llm_math_chain,
    name = "Calculator",
    description = "Useful for when you need to do mathematical calculations. This tool is only used for mathematics question"
)

In [19]:
tools = [search_tool, wiki_tool, math_tool]

In [20]:
# Base template
template_with_history = """Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a compasionate medical professional when giving your final answer. If the condition is serious advise they speak to a doctor.

Previous conversation history:
{history}

New question: {input}
{agent_scratchpad}"""

# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [21]:
prompt = CustomPromptTemplate(
    template=template_with_history,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps", "history"]
)

In [22]:
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)


output_parser = CustomOutputParser()

In [23]:
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]

In [24]:
memory=ConversationBufferWindowMemory(k=2)

In [25]:
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)
# agent = initialize_agent(
#     llm = llm,
#     tools = tools,
#     agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
#     verbose = True
# )

In [26]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, 
                                                    tools=tools, 
                                                    verbose=True,
                                                   memory=memory)

In [27]:
q1 = "How can I treat a spained ankle?"
q2 = "What are the causes of skin cancer?"
q3 = "What are the foods to avoid during pregnancy?"
q4 = "I am having flu. What should I do now?"

In [29]:
agent_executor.run(q4)



> Entering new AgentExecutor chain...
Thought: It is important to address the flu symptoms and prevent further complications.
Action: Search NIH
Action Input: "flu treatment"

Observation:Antiviral Treatment of Influenza When Influenza Viruses and SARS-CoV-2 Are Cocirculating. Antiviral treatment for influenza is the same for all patients regardless of SARS-CoV-2 coinfection (AIII). For information on using antiviral drugs to treat influenza in hospitalized and nonhospitalized patients, see the CDC and IDSA recommendations. Credit: NIAID. Seasonal influenza, or flu, is a highly contagious respiratory disease caused by any of several human influenza viruses that circulate globally and cause annual outbreaks of varying severity. Influenza viruses infect the nose, throat, and lungs and produce symptoms that include sudden fever, extreme fatigue, coughing, chills, and ... The prevention and treatment of influenza are best done with an interprofessional team that includes a nurse, nurse p

'It is important to seek medical attention and start antiviral treatment for influenza within 48 hours of symptom onset, even if a rapid test is not available. This can help reduce the severity and duration of the illness and prevent further complications. Remember to also get the annual flu vaccine to protect yourself and others from the flu. If you have any concerns or symptoms, please speak to a doctor for proper diagnosis and treatment.'